Это для Jupiter Notebook

# Цель исследования:

Как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? Возьмите данные для Москвы и Петербурга. Сравните, что и в каком режиме слушают их жители.

## Этап 1. Получение данных

In [3]:
# импортируем библиотеку pandas
import pandas as pd

In [4]:
# считываем файл с данными и сохраняем его в переменной df
df = pd.read_csv('music_project.csv')

In [5]:
# просматриваем первые 10 строк, проверяем, что отображается нормально
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [6]:
# общая информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


### Выводы

1. Строка таблицы содержит информацию об одной композиции (с указанием ее названия, исполнителя и жанра), которую слушали в определенном городе, в определенное время и в определеный день недели.
2. В столбцах разное количество значений. Это значит, что в таблице есть нулевые значения. Необходимо сделать так, чтобы нулевых значений не было.
3. У таблицы некорректные названия столбцов: присутствуют пробелы, используются прописные и строчные буквы, названия не всегда информативны. Необходимо поменять названия столбцов, исключив эти проблемы.
4. Для достижения цели исследования наиболее важны столбцы city, time, day и genre.

## Этап 2. Предобработка данных

In [7]:
# получаем перечень названий столбцов
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [8]:
# переименовываем столбы
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], axis='columns', inplace=True)

In [9]:
# проверяем новые названия столбцов
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

In [10]:
# проверим, есть ли пропуски в таблице
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [11]:
# заменяем пропуски в столбцах track_name и artist_name на 'unknown'
df['track_name'] = df['track_name'].fillna('unknown')
df['artist_name'] = df['artist_name'].fillna('unknown')

In [12]:
# проверяем
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [13]:
# удаляем записи, для которые не прописан genre_name - нам они не нужны,
# т.к. записи без genre_name не обладают достаточной информацией для участия в анализе
df = df.dropna()

In [14]:
# проверка
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

In [15]:
# проверка, остались ли пустые значения
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63881 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      63881 non-null  object
 1   track_name   63881 non-null  object
 2   artist_name  63881 non-null  object
 3   genre_name   63881 non-null  object
 4   city         63881 non-null  object
 5   time         63881 non-null  object
 6   weekday      63881 non-null  object
dtypes: object(7)
memory usage: 3.9+ MB


In [16]:
# ищем дубликаты
df.duplicated().sum()

3755

In [17]:
# удаляем дубликаты, обновляем индексы
df = df.drop_duplicates().reset_index(drop=True)

In [18]:
# проверка
df.duplicated().sum()

0

Уберем неявные дубликаты в списке жанров. Неявные дубликаты - это когда один и тот же жанр записан под разными названиями.

In [19]:
# сохраняем список уникальных значений стобца с жанрами
genres_list = df['genre_name'].unique()

In [20]:
# Создаем функцию, которая будет искать неявные дубликаты в списке жанров genres_list.
# Функция возвращает, есть ли искомое значение в списке (1), или его нет (0)
def find_genre(genre):
    count = 0
    for i in genres_list:
        if i == genre:
            count += 1
    return count

Проверяем наличие в списке genres_list различных названий жанра hiphop

In [21]:
find_genre('hip')

1

In [22]:
find_genre('hop')

0

In [23]:
find_genre('hip-hop')

0

In [24]:
# создаем функцию, которая будет заменять неправильное название жанра на правильное
def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong, 'hiphop')

In [25]:
# заменяем неправильное значение на правильное
find_hip_hop(df, 'hip')

In [26]:
# проверка
df[df['genre_name'] == 'hip']['genre_name'].count()

0

In [27]:
# обновленная информация по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


### Выводы

1. Изменены названия столбцов. Теперь они корректны и удобны в использовании.
2. В результате предобработки данных были частично заменены нулевые значения.
3. Данные, которые не могут быть использованы при анализы, либо использование которых не окажет влияния на результат, были удалены (строки с нулевым значением в поле genre_name).
4. Удалены дубликаты строк.
5. Оптимизирован список жанров (исключены неявные дубликаты).
6. Для достижения цели исследования наиболее важны столбцы city, time, weekday и genre_name.

## Этап 3. Анализ данных

### Вопрос 1: Действительно ли музыку в разных городах слушают по-разному?

Сравним, сколько времени включена музыка у жителей этих двух городов по понедельникам, средам и пятницам.

In [28]:
# Сгруппируем данные по городам и посчитаем, сколько в каждом городе прослушено треков с указанным жанром.
df.groupby('city')['genre_name'].count()




city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

In [29]:
# сгруппируем данные по дням недели и посчитаем, сколько прослушано треков в понедельник, среду и пятницу
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

In [33]:
# создаем функцию, которая бы подсчитывала количество прослушанных треков в зависимости от города и дня недели
def number_tracks(df, day, city):
    return df[(df['weekday'] == day) & (df['city'] == city)]['genre_name'].count()

In [35]:
# создадим таблицу с количеством прослушиваний в день недели по городам
data = [['Moscow', number_tracks(df, 'Monday', 'Moscow'), number_tracks(df, 'Wednesday', 'Moscow'), number_tracks(df, 'Friday', 'Moscow')], ['Saint-Petersburg', number_tracks(df, 'Monday', 'Saint-Petersburg'), number_tracks(df, 'Wednesday', 'Saint-Petersburg'), number_tracks(df, 'Friday', 'Saint-Petersburg')]]
columns = ['city', 'monday', 'wednesday', 'friday']
pd.DataFrame(data = data, columns = columns)

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


### Выводы

1. Общее количество прослушиваний в Санкт-Петербурге меньше, чем в Москве, но в Санкт-Петербурге в принципе количество пользователей Яндекс.Музыки ниже, чем в Москве, поэтому нельзя говорить, что пользовали в Москве более активны.
2. В Москве в понедельник и пятницу практически одинаковое количество прослушиваний. При этом в среду количество прослушиваний снижается.
3. В Санкт-Петербурге картина обратная: в среду количество прослушиваний больше, чем в понедельник и пятницу. Но в понедельник и пятницу количество прослушиваний, также как и в Москве, почти одинаковое.

### Вопрос 2: Утро понедельника и вечер пятницы — разная или одна и та же музыка?

Узнаем, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером.

In [38]:
# создаем отдельные таблицы по Москве и Санкт-Петербургу
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == "Saint-Petersburg"]

In [41]:
# проверяем Москву
df[df['city'] == 'Moscow'].head()

,user_id,track_name,artist_name,genre_name,city,time,weekday
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


In [42]:
# проверяем Санкт-Петербург
df[df['city'] == "Saint-Petersburg"].head()

,user_id,track_name,artist_name,genre_name,city,time,weekday
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


In [43]:
# создаем функцию, которая будет выдавать 10 наиболее популярных жанров в определенное время определенного дня недели
def genre_weekday(df, day, time1, time2):
    # получаем таблицу со всеми строками, удовлетворяющими условиям day, time1 и time2
    genre_list = df[(df['weekday'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    # считаем количество прослушиваний каждого жанра, группируем, сортируем и выводим первые 10
    return genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)

In [55]:
# Москва, утро понедельника
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [56]:
# Санкт-Петербург, утро понедельника
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [59]:
# Москва, вечер пятницы
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [60]:
# Санкт-Петербург, вечер пятницы
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

### Выводы

1. Утром понедельника и вечером пятницы первая пятерка жанров совпадает у Москвы и Санкт-Петербурга.
2. Утром понедельника во второй пятерке в Москве присутствует world, а в Санкт-Петербурге - jazz.
3. Вечером пятницы в Москве присутствет ruspop, а в Санкт-Петербурге - по прежнему jazz.
4. Таким образом, отличия в музыкальных пристастиях между Москвой и Санкт-Петербургом не такие уж существенные.
5. Топ-5 жанров совпадают, но во второй пятерке немного проявляются индивидуальные черты.
6. В Москве топ-10 жанров утром понедельника и вечером пятницы полностью совпадают.
7. В Санкт-Петербурге утром присутствует ruspop, а вечером - world.
8. Можно сделать вывода, что отличия между утром понедельника и вечером пятницы также несущественные.

### Вопрос 3: Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Проверим гипотезу о том, что Питер - про рэп, а Москва - про попсу.

In [61]:
# посмотрим на топ-10 жанров в Москве
moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [62]:
# посмотрим на топ-10 жанров в Санкт-Петербурге
spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

### Выводы

1. Среди наиболее популярных жанров в Москве, помимо pop, присутствет ruspop, так что предположение, что в Москве больше слушают поп-музыку похоже на правду.
2. Но рэп занимает в обоих городах примеро одинаковые позиции, так что гипотеза о большей популярности рэпа в Санкт-Петербутге не получила подтверждения.

## Этап 4. Результаты исследования

Рабочие гипотезы:
- музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;
- списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;
- население двух городов предпочитает разные музыкальные жанры.

### Общие результаты

1. Музыку в Москве и Санкт-Петербурге действительно слушают в разном режиме. В Москве больше музыки слушают в начале и конце рабочей недели, а в Санкт-Петербурге - в середине.
2. Отличия между утром понедельника и вечером пятницы по обоим городам несущественные.
3. Топ-10 наиболе популярных жанров по обоим городам тоже отличается несущественно. Наиболее популярным жанром в обоих городах является поп-музыка.